In [ ]:
!pip install torch-xla -f https://storage.googleapis.com/tpu-pytorch/wheels/tpuvm/torch_xla-1.12-cp37-cp37m-linux_x86_64.whl

Looking in links: https://storage.googleapis.com/tpu-pytorch/wheels/tpuvm/torch_xla-1.12-cp37-cp37m-linux_x86_64.whl
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 MB 12.4 MB/s eta 0:00:00:00:0100:01


In [ ]:
!pip install torch transformers pandas scikit-learn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
sinhala_fake1 = pd.read_csv('/content/drive/MyDrive/sinhala/sinhala_fake.csv')
sinhala_fake2 = pd.read_excel('/content/drive/MyDrive/sinhala/sinhala_fake_2.xlsx')
sinhala_real = pd.read_excel('/content/drive/MyDrive/sinhala/sinhala_real.xlsx')

In [ ]:
sinhala_fake = pd.concat([sinhala_fake1.drop('title', axis=1), sinhala_fake2.rename(columns={'title_sinhala': 'text'})], axis=0).reset_index(drop=True)

In [ ]:
sinhala_real.drop(['id','news_url','title'], axis=1, inplace=True)
sinhala_real.rename(columns={'title_sinhala': 'text'}, inplace=True)

In [ ]:
sinhala_fake['label'] = 1
sinhala_real['label'] = 0

In [ ]:
sinhala_news_df = pd.concat([sinhala_fake, sinhala_real], axis=0).reset_index(drop=True)

In [ ]:
sinhala_news_df.head(5)

,text,label
0,Miley Cyrus සහ Liam Hemsworth රහසේ විවාහ වුණාද?,1
1,පැරිස් ජැක්සන් සහ කාරා ඩෙලිවින්ග්න් ගැලපෙන ඇඳු...,1
2,ඩොනල්ඩ් ට්‍රම්ප්ට විරෝධය පළ කරමින් ප්‍රසිද්ධ ප...,1
3,සින්ඩි ක්‍රෝෆර්ඩ්ගේ දියණිය Kaia Gerber හැරී ස්...,1
4,2018 ඔස්කාර් නාමයෝජනා සම්පූර්ණ ලැයිස්තුව - විව...,1


In [ ]:
# Assuming your dataset is in a DataFrame named 'train_data' and the label column is named 'label'
label_counts = sinhala_news_df['label'].value_counts()
print("Class distribution in training data:")
print(label_counts)

Class distribution in training data:
label
0    5999
1    5331
Name: count, dtype: int64


In [ ]:
import pandas as pd
import re

# Define the cleaning function
def clean_text(text):
    if not isinstance(text, str):  # Check if the value is not a string
        return ""
    text = re.sub(r"http\S+", "", text)  # Remove URLs
    text = re.sub(r"[^අ-෴a-zA-Z\s]", "", text)  # Keep Sinhala and English letters
    text = re.sub(r"\s+", " ", text).strip()  # Remove extra spaces
    return text

# Apply the cleaning function safely
sinhala_news_df['text'] = sinhala_news_df['text'].fillna("").apply(clean_text)

In [ ]:
sinhala_news_df.head(5)

,text,label
0,Miley Cyrus සහ Liam Hemsworth රහසේ විවාහ වුණාද,1
1,පැරිස් ජැක්සන් සහ කාරා ඩෙලිවින්ග්න් ගැලපෙන ඇඳු...,1
2,ඩොනල්ඩ් ට්රම්ප්ට විරෝධය පළ කරමින් ප්රසිද්ධ පුද...,1
3,සින්ඩි ක්රෝෆර්ඩ්ගේ දියණිය Kaia Gerber හැරී ස්ට...,1
4,ඔස්කාර් නාමයෝජනා සම්පූර්ණ ලැයිස්තුව විවිධත්වය,1


In [ ]:
sinhala_news_df.shape

(11330, 2)

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

# Load SinBERT-large
tokenizer = AutoTokenizer.from_pretrained("NLPC-UOM/SinBERT-large")
model = AutoModelForMaskedLM.from_pretrained("NLPC-UOM/SinBERT-large")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/722 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.50M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.15M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/504M [00:00<?, ?B/s]

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "NLPC-UOM/SinBERT-large",
    num_labels=2  # Binary classification (FAKE, REAL)
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at NLPC-UOM/SinBERT-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(
    sinhala_news_df['text'], sinhala_news_df['label'], test_size=0.2, random_state=42
)

In [ ]:
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=512, return_tensors="pt")
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=512, return_tensors="pt")

In [ ]:
import torch

class NewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "labels": torch.tensor(self.labels[idx], dtype=torch.long)
        }

# Create datasets
train_dataset = NewsDataset(train_encodings, train_labels.tolist())
val_dataset = NewsDataset(val_encodings, val_labels.tolist())


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    save_strategy="epoch",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,0.606000,0.542932
2,0.483300,0.545263
3,0.412600,0.557315


TrainOutput(global_step=1701, training_loss=0.4868096380216945, metrics={'train_runtime': 2655.3087, 'train_samples_per_second': 10.241, 'train_steps_per_second': 0.641, 'total_flos': 7154515817349120.0, 'train_loss': 0.4868096380216945, 'epoch': 3.0})

In [ ]:
preds = trainer.predict(val_dataset)
predicted_labels = torch.argmax(torch.tensor(preds.predictions), axis=1)

from sklearn.metrics import classification_report

print(classification_report(val_labels, predicted_labels, target_names=["FAKE", "REAL"]))


              precision    recall  f1-score   support

        FAKE       0.76      0.74      0.75      1172
        REAL       0.73      0.75      0.74      1094

    accuracy                           0.75      2266
   macro avg       0.75      0.75      0.75      2266
weighted avg       0.75      0.75      0.75      2266



In [ ]:
model.save_pretrained("/content/drive/MyDrive/sinhala/sinhala_fake_news_model")
tokenizer.save_pretrained("/content/drive/MyDrive/sinhala/sinhala_fake_news_tokenizer")

('/content/drive/MyDrive/sinhala/sinhala_fake_news_tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/sinhala/sinhala_fake_news_tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/sinhala/sinhala_fake_news_tokenizer/vocab.json',
 '/content/drive/MyDrive/sinhala/sinhala_fake_news_tokenizer/merges.txt',
 '/content/drive/MyDrive/sinhala/sinhala_fake_news_tokenizer/added_tokens.json',
 '/content/drive/MyDrive/sinhala/sinhala_fake_news_tokenizer/tokenizer.json')